In [4]:
import pandas as pd
import os

In [14]:
root_path = os.path.dirname(os.path.dirname(os.getcwd()))  # go 3 back, on the same level with the "data" directory

datasets_path = os.path.join(root_path, "data", "equate")
print(datasets_path)

/Users/ioanamazilu/PycharmProjects/quant_nli/data/equate


In [17]:
os.makedirs(os.path.join(datasets_path, "01_duplicates"), exist_ok=True)
os.makedirs(os.path.join(datasets_path, "01_deduplicated"), exist_ok=True)
for dataset in ["AWPNLI", "NewsNLI", "RedditNLI", "RTE_Quant", "StressTest"]:
    print(dataset)
    df = pd.read_csv(os.path.join(datasets_path, f"{dataset}.csv"))
    print(f"Dataset size pre-deduplication: {df.shape[0]}")

    duplicates = df[df.duplicated(subset=["premise", "hypothesis"])].shape[0]
    print(f"{duplicates} duplicates")
    if duplicates > 0:
        duplicates_add_label = df[df.duplicated(subset=["premise", "hypothesis", "label"])].shape[0]
        print(f"{duplicates_add_label} duplicates (with label)")

        # Turn the premise and hypothesis to lowercase, to ensure we do a case-insensitive check for duplicates as well
        df["premise_lower"] = df["premise"].str.lower()
        df["hypothesis_lower"] = df["hypothesis"].str.lower()

        duplicates_lowercase = df[df.duplicated(subset=["premise_lower", "hypothesis_lower"])].shape[0]
        print(f"{duplicates_lowercase} duplicates (lowercase)")

        duplicates = df[df.duplicated(subset=["premise_lower", "hypothesis_lower"])][["premise", "hypothesis", "label", "sample_index"]]

        duplicates.to_csv(os.path.join(datasets_path, "01_duplicates", f"duplicates_{dataset}.csv"), index=False)

        deduplicated_df = df.drop_duplicates(subset=["premise_lower", "hypothesis_lower"], ignore_index=True)

        deduplicated_df[["premise", "hypothesis", "label", "sample_index"]].to_csv(os.path.join(datasets_path, "01_deduplicated", f"{dataset}.csv"), index=False)

        print(f"Dataset size post-deduplication: {deduplicated_df.shape[0]}")
    else:
        df[["premise", "hypothesis", "label", "sample_index"]].to_csv(os.path.join(datasets_path, "01_deduplicated", f"{dataset}.csv"), index=False)

AWPNLI
Dataset size pre-deduplication: 722
0 duplicates
NewsNLI
Dataset size pre-deduplication: 968
5 duplicates
5 duplicates (with label)
5 duplicates (lowercase)
Dataset size post-deduplication: 963
RedditNLI
Dataset size pre-deduplication: 250
3 duplicates
3 duplicates (with label)
3 duplicates (lowercase)
Dataset size post-deduplication: 247
RTE_Quant
Dataset size pre-deduplication: 166
1 duplicates
1 duplicates (with label)
1 duplicates (lowercase)
Dataset size post-deduplication: 165
StressTest
Dataset size pre-deduplication: 7596
643 duplicates
643 duplicates (with label)
649 duplicates (lowercase)
Dataset size post-deduplication: 6947
